# AnimalCLEF 2025: ResNet and DINOv2 Implementation

This notebook implements both ResNet and DINOv2 models for the AnimalCLEF 2025 competition, which focuses on individual animal identification for three species:
- 🐢 Loggerhead sea turtles (Zakynthos, Greece)
- 🦎 Salamanders (Czech Republic)
- 🐆 Eurasian lynxes (Czech Republic)

The goal is to determine whether an animal in an image is new (not present in the training dataset) or known (in which case, its identity must be provided).

## 1. Environment Setup and Dependencies Installation

In [67]:
EPOCH_NUM = 5 # 15, 30
FINETUNE_RESNET = True
FINETUNE_DINO = False

In [68]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler, WeightedRandomSampler
from torchvision import transforms, models
from PIL import Image, ImageFilter
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoImageProcessor
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random
import time
import copy
from collections import defaultdict

In [ ]:
# Install required packages
!pip install -q git+https://github.com/WildlifeDatasets/wildlife-datasets@develop
!pip install -q git+https://github.com/WildlifeDatasets/wildlife-tools
!pip install -q timm transformers albumentations

In [70]:
# Set data path
DATA_PATH = 'animal-clef-2025'
# Create directory for saving models
MODEL_SAVE_PATH = '/home/intern/Yu/animal-clef-2025/training'
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

In [ ]:
# Check if running in Kaggle or Colab
IN_KAGGLE = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
IN_COLAB = 'google.colab' in sys.modules

if IN_KAGGLE:
    print("Running in Kaggle")
    # Set data path
    DATA_PATH = '/kaggle/input/animal-clef-2025'
    MODEL_SAVE_PATH = '/kaggle/working/models'
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
elif IN_COLAB:
    print("Running in Google Colab")

    # Upload kaggle.json if needed
    from google.colab import files
    try:
        files.upload()  # Upload kaggle.json
        !mkdir -p ~/.kaggle
        !cp kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
    except:
        print("Please upload your kaggle.json file to access competition data")

    # Download competition data
    !kaggle competitions download -c animal-clef-2025
    !unzip -q animal-clef-2025.zip -d animal-clef-2025

else:
    print("Running in local environment")
    # Set appropriate path for your environment
    DATA_PATH = './animal-clef-2025'
    MODEL_SAVE_PATH = './models'
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

In [72]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

Using device: cuda


## 2. Data Loading and Preprocessing

In [73]:
# Load metadata
metadata_path = os.path.join(DATA_PATH, 'metadata.csv')
metadata = pd.read_csv(metadata_path)

# Display basic information about the metadata
print(f"Metadata shape: {metadata.shape}")
metadata.head()

Metadata shape: (15209, 8)


,image_id,identity,path,date,orientation,species,split,dataset
0,0,LynxID2025_lynx_37,images/LynxID2025/database/000f9ee1aad063a4485...,NaN,right,lynx,database,LynxID2025
1,1,LynxID2025_lynx_37,images/LynxID2025/database/0020edb6689e9f78462...,NaN,left,lynx,database,LynxID2025
2,2,LynxID2025_lynx_49,images/LynxID2025/database/003152e4145b5b69400...,NaN,left,lynx,database,LynxID2025
3,3,NaN,images/LynxID2025/query/003b89301c7b9f6d18f722...,NaN,back,lynx,query,LynxID2025
4,4,LynxID2025_lynx_13,images/LynxID2025/database/003c3f82011e9c3f849...,NaN,right,lynx,database,LynxID2025


In [74]:
# Check for missing values in the metadata
missing_values = metadata.isnull().sum()
print("Missing values in each column:")
print(missing_values)

# Check unique values in categorical columns
print("\nUnique values in 'species' column:")
print(metadata['species'].unique())

print("\nUnique values in 'orientation' column:")
print(metadata['orientation'].unique())

print("\nUnique values in 'dataset' column:")
print(metadata['dataset'].unique())

print("\nUnique values in 'split' column:")
print(metadata['split'].unique())

Missing values in each column:
image_id          0
identity       2135
path              0
date           3907
orientation     703
species        1388
split             0
dataset           0
dtype: int64

Unique values in 'species' column:
['lynx' nan 'salamander' 'loggerhead turtle']

Unique values in 'orientation' column:
['right' 'left' 'back' 'front' 'unknown' 'top' 'bottom' 'topright'
 'topleft' nan 'down']

Unique values in 'dataset' column:
['LynxID2025' 'SalamanderID2025' 'SeaTurtleID2022']

Unique values in 'split' column:
['database' 'query']


### 2.1 Handling NaN Values in Metadata

Based on the analysis above, we need to handle several types of missing values in the metadata:

1. **Empty identity fields for query images**: This is expected since these are the images we need to identify.
2. **Missing date values**: We'll fill these with a default value since the date might not be critical for our model.
3. **Unknown orientation values**: We'll treat 'unknown' as a separate category in our encoding.

In [75]:
# Create a copy of the metadata for preprocessing
metadata_processed = metadata.copy()

# 1. Handle missing identity values for query images
# For query images, we'll keep the identity as NaN since these are what we need to predict
# We'll verify that all query images have NaN identity and all database images have an identity
query_missing_identity = metadata_processed[metadata_processed['split'] == 'query']['identity'].isnull().sum()
query_total = metadata_processed[metadata_processed['split'] == 'query'].shape[0]
database_has_identity = metadata_processed[metadata_processed['split'] == 'database']['identity'].notnull().sum()
database_total = metadata_processed[metadata_processed['split'] == 'database'].shape[0]

print(f"Query images with missing identity: {query_missing_identity} out of {query_total}")
print(f"Database images with identity: {database_has_identity} out of {database_total}")

# 2. Handle missing date values
# Fill missing dates with a default value 'unknown_date'
metadata_processed['date'] = metadata_processed['date'].fillna('unknown_date')

# 3. Handle orientation values
# First, check the distribution of orientation values
orientation_counts = metadata_processed['orientation'].value_counts(dropna=False)
print("\nOrientation distribution:")
print(orientation_counts)

# Fill missing orientation values with 'unknown'
metadata_processed['orientation'] = metadata_processed['orientation'].fillna('unknown')

# Verify that we've handled all missing values except for identity in query images
missing_after = metadata_processed.isnull().sum()
print("\nMissing values after processing:")
print(missing_after)

Query images with missing identity: 2135 out of 2135
Database images with identity: 13074 out of 13074

Orientation distribution:
orientation
right       4655
left        4231
top         2009
topright    1490
topleft     1354
NaN          703
front        318
unknown      245
back         167
down          34
bottom         3
Name: count, dtype: int64

Missing values after processing:
image_id          0
identity       2135
path              0
date              0
orientation       0
species        1388
split             0
dataset           0
dtype: int64


### 2.2 Encoding Categorical Features

In [76]:
# Encode categorical features
from sklearn.preprocessing import LabelEncoder

# Encode species
species_encoder = LabelEncoder()
metadata_processed['species_encoded'] = species_encoder.fit_transform(metadata_processed['species'])

# Encode orientation
orientation_encoder = LabelEncoder()
metadata_processed['orientation_encoded'] = orientation_encoder.fit_transform(metadata_processed['orientation'])

# Encode dataset
dataset_encoder = LabelEncoder()
metadata_processed['dataset_encoded'] = dataset_encoder.fit_transform(metadata_processed['dataset'])

# For identity, we'll create a mapping for database images only
# Query images will be handled separately during prediction
database_identities = metadata_processed[metadata_processed['split'] == 'database']['identity'].dropna().unique()
identity_encoder = LabelEncoder()
identity_encoder.fit(list(database_identities) + ['new_individual'])  # Add 'new_individual' as a class

# Create a new column for encoded identity, but only fill it for database images
metadata_processed['identity_encoded'] = np.nan
mask = metadata_processed['split'] == 'database'
metadata_processed.loc[mask, 'identity_encoded'] = identity_encoder.transform(metadata_processed.loc[mask, 'identity'])

# Display the processed metadata
print("Processed metadata:")
metadata_processed.head()

Processed metadata:


,image_id,identity,path,date,orientation,species,split,dataset,species_encoded,orientation_encoded,dataset_encoded,identity_encoded
0,0,LynxID2025_lynx_37,images/LynxID2025/database/000f9ee1aad063a4485...,unknown_date,right,lynx,database,LynxID2025,1,5,0,29.0
1,1,LynxID2025_lynx_37,images/LynxID2025/database/0020edb6689e9f78462...,unknown_date,left,lynx,database,LynxID2025,1,4,0,29.0
2,2,LynxID2025_lynx_49,images/LynxID2025/database/003152e4145b5b69400...,unknown_date,left,lynx,database,LynxID2025,1,4,0,40.0
3,3,NaN,images/LynxID2025/query/003b89301c7b9f6d18f722...,unknown_date,back,lynx,query,LynxID2025,1,0,0,NaN
4,4,LynxID2025_lynx_13,images/LynxID2025/database/003c3f82011e9c3f849...,unknown_date,right,lynx,database,LynxID2025,1,5,0,11.0


### 2.3 Creating Dataset Classes

In [77]:
# Define image transformations for ResNet
resnet_train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

resnet_val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define image transformations for DINOv2 using Albumentations
dinov2_train_transform = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

dinov2_val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Create dataset class for ResNet
class AnimalDatasetResNet(Dataset):
    def __init__(self, metadata, data_path, transform=None, is_query=False):
        self.metadata = metadata
        self.data_path = data_path
        self.transform = transform
        self.is_query = is_query

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        img_path = os.path.join(self.data_path, row['path'])

        # Load and transform image
        try:
            img = Image.open(img_path).convert('RGB')
            if self.transform:
                img = self.transform(img)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a blank image if there's an error
            img = torch.zeros((3, 224, 224))

        # For query images, we only return the image and metadata
        if self.is_query:
            return {
                'image': img,
                'image_id': row['image_id'],
                'path': row['path'],
                'species': row['species_encoded'],
                'orientation': row['orientation_encoded'],
                'dataset': row['dataset_encoded']
            }

        # For database images, we also return the identity
        return {
            'image': img,
            'identity': row['identity_encoded'],
            'image_id': row['image_id'],
            'path': row['path'],
            'species': row['species_encoded'],
            'orientation': row['orientation_encoded'],
            'dataset': row['dataset_encoded']
        }

# Create dataset class for DINOv2
class AnimalDatasetDINOv2(Dataset):
    def __init__(self, metadata, data_path, transform=None, is_query=False):
        self.metadata = metadata
        self.data_path = data_path
        self.transform = transform
        self.is_query = is_query

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        img_path = os.path.join(self.data_path, row['path'])

        # Load image
        try:
            img = np.array(Image.open(img_path).convert('RGB'))
            if self.transform:
                transformed = self.transform(image=img)
                img = transformed['image']
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a blank image if there's an error
            img = torch.zeros((3, 224, 224))

        # For query images, we only return the image and metadata
        if self.is_query:
            return {
                'image': img,
                'image_id': row['image_id'],
                'path': row['path'],
                'species': row['species_encoded'],
                'orientation': row['orientation_encoded'],
                'dataset': row['dataset_encoded']
            }

        # For database images, we also return the identity
        return {
            'image': img,
            'identity': row['identity_encoded'],
            'image_id': row['image_id'],
            'path': row['path'],
            'species': row['species_encoded'],
            'orientation': row['orientation_encoded'],
            'dataset': row['dataset_encoded']
        }

/home/intern/Yu/Animal-Clef-2025/animalvenv/lib/python3.11/site-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


### 2.4 Preparing Data Loaders

In [78]:
# Split database data into train and validation sets
database_data = metadata_processed[metadata_processed['split'] == 'database']
query_data = metadata_processed[metadata_processed['split'] == 'query']

# Check identity distribution to identify classes with only one sample
identity_counts = database_data['identity'].value_counts()
print(f"Number of identities: {len(identity_counts)}")
print(f"Identities with only one sample: {sum(identity_counts == 1)}")

# Filter out identities with only one sample for stratification
# These will be added to the training set directly
single_sample_identities = identity_counts[identity_counts == 1].index.tolist()
multi_sample_data = database_data[~database_data['identity'].isin(single_sample_identities)]
single_sample_data = database_data[database_data['identity'].isin(single_sample_identities)]

print(f"Data with multiple samples per identity: {len(multi_sample_data)}")
print(f"Data with single sample per identity: {len(single_sample_data)}")

# Stratify only the data with multiple samples per identity
if len(multi_sample_data) > 0:
    train_data, val_data = train_test_split(
        multi_sample_data,
        test_size=0.2,
        random_state=SEED,
        stratify=multi_sample_data['identity']
    )

    # Add single sample data to training set
    train_data = pd.concat([train_data, single_sample_data])
else:
    # If there's no multi-sample data, use a simple random split
    train_data, val_data = train_test_split(
        database_data,
        test_size=0.2,
        random_state=SEED
    )

print(f"Training data size: {len(train_data)}")
print(f"Validation data size: {len(val_data)}")
print(f"Query data size: {len(query_data)}")

# Create datasets for ResNet
train_dataset_resnet = AnimalDatasetResNet(
    train_data,
    DATA_PATH,
    transform=resnet_train_transform,
    is_query=False
)

val_dataset_resnet = AnimalDatasetResNet(
    val_data,
    DATA_PATH,
    transform=resnet_val_transform,
    is_query=False
)

query_dataset_resnet = AnimalDatasetResNet(
    query_data,
    DATA_PATH,
    transform=resnet_val_transform,
    is_query=True
)

# Create datasets for DINOv2
train_dataset_dinov2 = AnimalDatasetDINOv2(
    train_data,
    DATA_PATH,
    transform=dinov2_train_transform,
    is_query=False
)

val_dataset_dinov2 = AnimalDatasetDINOv2(
    val_data,
    DATA_PATH,
    transform=dinov2_val_transform,
    is_query=False
)

query_dataset_dinov2 = AnimalDatasetDINOv2(
    query_data,
    DATA_PATH,
    transform=dinov2_val_transform,
    is_query=True
)

# Create data loaders
BATCH_SIZE = 32
NUM_WORKERS = 4

train_loader_resnet = DataLoader(
    train_dataset_resnet,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

val_loader_resnet = DataLoader(
    val_dataset_resnet,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

query_loader_resnet = DataLoader(
    query_dataset_resnet,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

train_loader_dinov2 = DataLoader(
    train_dataset_dinov2,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

val_loader_dinov2 = DataLoader(
    val_dataset_dinov2,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

query_loader_dinov2 = DataLoader(
    query_dataset_dinov2,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

Number of identities: 1102
Identities with only one sample: 317
Data with multiple samples per identity: 12757
Data with single sample per identity: 317
Training data size: 10522
Validation data size: 2552
Query data size: 2135


## 3. Model Implementation

### 3.1 ResNet Model

In [79]:
class ResNetModel(nn.Module):
    def __init__(self, num_classes, embedding_dim=512):
        super(ResNetModel, self).__init__()
        # Load pre-trained ResNet50
        self.backbone = models.resnet50(pretrained=True)
        in_features = self.backbone.fc.in_features

        # Replace the final fully connected layer
        self.backbone.fc = nn.Identity()

        # Add embedding layer
        self.embedding = nn.Sequential(
            nn.Linear(in_features, embedding_dim),
            nn.BatchNorm1d(embedding_dim),
            nn.ReLU()
        )

        # Add classifier head
        self.classifier = nn.Linear(embedding_dim, num_classes)

    def forward(self, x, return_embeddings=False):
        # Extract features from backbone
        features = self.backbone(x)

        # Get embeddings
        embeddings = self.embedding(features)

        if return_embeddings:
            return embeddings

        # Get class predictions
        logits = self.classifier(embeddings)

        return logits, embeddings

### 3.2 DINOv2 Model

In [80]:
class DINOv2Model(nn.Module):
    def __init__(self, num_classes, embedding_dim=512):
        super(DINOv2Model, self).__init__()
        # Load pre-trained DINOv2 model
        self.processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
        self.backbone = AutoModel.from_pretrained("facebook/dinov2-base")

        # Get the output dimension of the backbone
        in_features = self.backbone.config.hidden_size

        # Add embedding layer
        self.embedding = nn.Sequential(
            nn.Linear(in_features, embedding_dim),
            nn.BatchNorm1d(embedding_dim),
            nn.ReLU()
        )

        # Add classifier head
        self.classifier = nn.Linear(embedding_dim, num_classes)

    def forward(self, x, return_embeddings=False):
        # Extract features from backbone
        outputs = self.backbone(x)
        features = outputs.last_hidden_state[:, 0]  # Use CLS token

        # Get embeddings
        embeddings = self.embedding(features)

        if return_embeddings:
            return embeddings

        # Get class predictions
        logits = self.classifier(embeddings)

        return logits, embeddings

### 3.3 Training Functions

In [81]:
def train_model(model, criterion, optimizer, scheduler, dataloader, num_epochs=10, model_name="model"):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    history = {'train_loss': [], 'train_acc': []}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train()
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for batch in tqdm(dataloader, desc=f"Training {model_name}"):
            inputs = batch['image'].to(device)
            labels = batch['identity'].long().to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            with torch.set_grad_enabled(True):
                outputs, embeddings = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        if scheduler is not None:
            scheduler.step()

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc.item())

        # Save the best model
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), os.path.join(MODEL_SAVE_PATH, f"{model_name}_best.pth"))

        # Save checkpoint every 5 epochs
        if (epoch + 1) % 5 == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': epoch_loss,
            }, os.path.join(MODEL_SAVE_PATH, f"{model_name}_epoch_{epoch+1}.pth"))

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

### 3.4 Evaluation Functions

In [82]:
def extract_embeddings(model, dataloader, model_name):
    model.eval()
    embeddings = []
    image_ids = []
    paths = []
    identities = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Extracting embeddings with {model_name}"):
            inputs = batch['image'].to(device)

            # Get embeddings
            if model_name == 'resnet':
                batch_embeddings = model(inputs, return_embeddings=True)
            else:  # dinov2
                batch_embeddings = model(inputs, return_embeddings=True)

            # Move to CPU and convert to numpy
            batch_embeddings = batch_embeddings.cpu().numpy()

            # Store embeddings and metadata
            embeddings.append(batch_embeddings)
            image_ids.extend(batch['image_id'].numpy())
            paths.extend(batch['path'])

            # Store identities if available (for database images)
            if 'identity' in batch:
                identities.extend(batch['identity'].numpy())

    # Concatenate embeddings
    embeddings = np.vstack(embeddings)

    # Create a dictionary with all information
    result = {
        'embeddings': embeddings,
        'image_ids': np.array(image_ids),
        'paths': np.array(paths)
    }

    if identities:
        result['identities'] = np.array(identities)

    return result

def evaluate_model(database_embeddings, query_embeddings, identity_encoder, threshold=0.5):
    # Normalize embeddings
    database_norm = database_embeddings['embeddings'] / np.linalg.norm(database_embeddings['embeddings'], axis=1, keepdims=True)
    query_norm = query_embeddings['embeddings'] / np.linalg.norm(query_embeddings['embeddings'], axis=1, keepdims=True)

    # Find nearest neighbors
    knn = NearestNeighbors(n_neighbors=1, metric='cosine')
    knn.fit(database_norm)
    distances, indices = knn.kneighbors(query_norm)

    # Convert distances to similarities (1 - distance)
    similarities = 1 - distances

    # Predict identities
    predicted_identities = []
    for i, (similarity, idx) in enumerate(zip(similarities.flatten(), indices.flatten())):
        if similarity >= threshold:
            # Known individual
            predicted_identities.append(database_embeddings['identities'][idx])
        else:
            # New individual
            predicted_identities.append(identity_encoder.transform(['new_individual'])[0])

    return np.array(predicted_identities), similarities.flatten()

def calculate_metrics(y_true, y_pred):
    # Calculate standard metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    # Calculate BAKS (Balanced Accuracy for Known Samples)
    # This is the accuracy for samples that are known individuals
    known_mask = y_true != identity_encoder.transform(['new_individual'])[0]
    if np.sum(known_mask) > 0:
        baks = accuracy_score(y_true[known_mask], y_pred[known_mask])
    else:
        baks = 0.0

    # Calculate BAUS (Balanced Accuracy for Unknown Samples)
    # This is the accuracy for samples that are new individuals
    unknown_mask = y_true == identity_encoder.transform(['new_individual'])[0]
    if np.sum(unknown_mask) > 0:
        baus = accuracy_score(y_true[unknown_mask], y_pred[unknown_mask])
    else:
        baus = 0.0

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'baks': baks,
        'baus': baus
    }

## 4. Model Training

In [83]:
# Upload RestNet model and history manually or use this to upload from local
#  from google.colab import files
from sklearn.preprocessing import LabelEncoder
import torch.serialization
import pickle

In [84]:
if not FINETUNE_RESNET:
    num_classes = len(identity_encoder.classes_)
    print(f"Number of classes: {num_classes}")

    # Initialize ResNet model
    resnet_model = ResNetModel(num_classes=num_classes, embedding_dim=512).to(device)

In [85]:
if FINETUNE_RESNET: 
    # uploaded = files.upload()  # Upload resnet_model.pth
    
    # Allowlist LabelEncoder
    torch.serialization.add_safe_globals({'LabelEncoder': LabelEncoder})

    # Now load checkpoint
    checkpoint = torch.load(f'resnet_model_history/resnet_model_{EPOCH_NUM}.pth', map_location=device, weights_only=False)

    # Extract encoder
    identity_encoder = checkpoint['identity_encoder']

    # Initialize model with correct number of classes
    num_classes = len(identity_encoder.classes_)
    resnet_model = ResNetModel(num_classes=num_classes, embedding_dim=512).to(device)

    # Initialize optimizer and scheduler
    resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=1e-4, weight_decay=1e-5)
    resnet_scheduler = optim.lr_scheduler.StepLR(resnet_optimizer, step_size=5, gamma=0.1)

    # Load model weights and training states
    resnet_model.load_state_dict(checkpoint['model_state_dict'])
    resnet_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    resnet_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    # # Load later
    # with open(f'resnet_model_history/resnet_history_{EPOCH_NUM}.pth', 'rb') as f:
    #     resnet_history = pickle.load(f)

    # print("✅ ResNet model and optimizer state loaded successfully!")

/home/intern/Yu/Animal-Clef-2025/animalvenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/intern/Yu/Animal-Clef-2025/animalvenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [86]:
# # Get number of classes (identities)
# num_classes = len(identity_encoder.classes_)
# print(f"Number of classes: {num_classes}")

# # Initialize ResNet model
# resnet_model = ResNetModel(num_classes=num_classes, embedding_dim=512).to(device)

# # Initialize optimizer and scheduler for ResNet
# resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=1e-4, weight_decay=1e-5)
# resnet_scheduler = optim.lr_scheduler.StepLR(resnet_optimizer, step_size=5, gamma=0.1)

# # Initialize loss function
# criterion = nn.CrossEntropyLoss()

# # Train ResNet model
# print("Training ResNet model...")
# resnet_model, resnet_history = train_model(
#     resnet_model,
#     criterion,
#     resnet_optimizer,
#     resnet_scheduler,
#     train_loader_resnet,
#     num_epochs=5,
#     model_name="resnet"
# )

# torch.save({
#     'model_state_dict': resnet_model.state_dict(),
#     'optimizer_state_dict': resnet_optimizer.state_dict(),
#     'scheduler_state_dict': resnet_scheduler.state_dict(),
#     'identity_encoder': identity_encoder  # if picklable
# }, 'resnet_model.pth')

# print("Model saved locally in Colab environment.")

# # Save history
# with open('resnet_history.pkl', 'wb') as f:
#     pickle.dump(resnet_history, f)

In [87]:
if not FINETUNE_DINO:
    dinov2_model = DINOv2Model(num_classes=num_classes, embedding_dim=512).to(device)

In [88]:
# Upload DINOv2 model and history manually or use this to upload from local
if FINETUNE_DINO:
    # uploaded = files.upload()  # Upload resnet_model.pth

    # Allowlist LabelEncoder
    torch.serialization.add_safe_globals({'LabelEncoder': LabelEncoder})

    # Now load checkpoint
    checkpoint = torch.load(f'dino_model_history/dinov2_history_{EPOCH_NUM}.pth', map_location=device, weights_only=False)

    # Extract encoder
    identity_encoder = checkpoint['identity_encoder']

    # Initialize model with correct number of classes
    num_classes = len(identity_encoder.classes_)
    dinov2_model = DINOv2Model(num_classes=num_classes, embedding_dim=512).to(device)

    # Initialize optimizer and scheduler
    dinov2_optimizer = optim.Adam(dinov2_model.parameters(), lr=1e-4, weight_decay=1e-5)
    dinov2_scheduler = optim.lr_scheduler.StepLR(dinov2_optimizer, step_size=5, gamma=0.1)

    # Load model weights and training states
    dinov2_model.load_state_dict(checkpoint['model_state_dict'])
    dinov2_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    dinov2_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    # Load later
    # with open(f'dino_model_history/dinov2_history_{EPOCH_NUM}.pkl', 'rb') as f:
    #     dinov2_history = pickle.load(f)

    # print("✅ DINOv2 model and optimizer state loaded successfully!")

In [89]:
# # Initialize DINOv2 model
# dinov2_model = DINOv2Model(num_classes=num_classes, embedding_dim=512).to(device)

# # Initialize optimizer and scheduler for DINOv2
# dinov2_optimizer = optim.Adam(dinov2_model.parameters(), lr=1e-5, weight_decay=1e-5)
# dinov2_scheduler = optim.lr_scheduler.StepLR(dinov2_optimizer, step_size=5, gamma=0.1)

# # Train DINOv2 model
# print("Training DINOv2 model...")
# dinov2_model, dinov2_history = train_model(
#     dinov2_model,
#     criterion,
#     dinov2_optimizer,
#     dinov2_scheduler,
#     train_loader_dinov2,
#     num_epochs=5,
#     model_name="dinov2"
# )

# # Save model state_dict
# torch.save({
#     'model_state_dict': dinov2_model.state_dict(),
#     'optimizer_state_dict': dinov2_optimizer.state_dict(),
#     'scheduler_state_dict': dinov2_scheduler.state_dict(),
#     'identity_encoder': identity_encoder,  # Optional: save encoder too
# }, 'dinov2_model_5.pth')

# print("DINOv2 model saved successfully!")

# # from google.colab import files
# # files.download('dinov2_model.pth')

# # Save history
# with open('dinov2_history_5.pkl', 'wb') as f:
#     pickle.dump(dinov2_history, f)

## 5. Model Evaluation

In [90]:
# Extract embeddings from validation set
print("Extracting embeddings from validation set...")
val_embeddings_resnet = extract_embeddings(resnet_model, val_loader_resnet, "resnet")

# Extract embeddings from database set (for prediction)
print("Extracting embeddings from database set...")
database_loader_resnet = DataLoader(
    AnimalDatasetResNet(database_data, DATA_PATH, transform=resnet_val_transform, is_query=False),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

database_embeddings_resnet = extract_embeddings(resnet_model, database_loader_resnet, "resnet")

# Extract embeddings from query set
print("Extracting embeddings from query set...")
query_embeddings_resnet = extract_embeddings(resnet_model, query_loader_resnet, "resnet")

Extracting embeddings from validation set...


Extracting embeddings with resnet: 100%|██████████| 80/80 [00:07<00:00, 11.37it/s]


Extracting embeddings from database set...


Extracting embeddings with resnet: 100%|██████████| 409/409 [00:44<00:00,  9.27it/s]


Extracting embeddings from query set...


Extracting embeddings with resnet: 100%|██████████| 67/67 [00:12<00:00,  5.34it/s]


In [91]:
# Extract embeddings from validation set
print("Extracting embeddings from validation set...")
val_embeddings_dinov2 = extract_embeddings(dinov2_model, val_loader_dinov2, "dinov2")

# Extract embeddings from database set (for prediction)
print("Extracting embeddings from database set...")

database_loader_dinov2 = DataLoader(
    AnimalDatasetDINOv2(database_data, DATA_PATH, transform=dinov2_val_transform, is_query=False),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

database_embeddings_dinov2 = extract_embeddings(dinov2_model, database_loader_dinov2, "dinov2")

# Extract embeddings from query set
print("Extracting embeddings from query set...")
query_embeddings_dinov2 = extract_embeddings(dinov2_model, query_loader_dinov2, "dinov2")

Extracting embeddings from validation set...


Extracting embeddings with dinov2: 100%|██████████| 80/80 [00:28<00:00,  2.81it/s]


Extracting embeddings from database set...


Extracting embeddings with dinov2: 100%|██████████| 409/409 [02:24<00:00,  2.83it/s]


Extracting embeddings from query set...


Extracting embeddings with dinov2: 100%|██████████| 67/67 [00:24<00:00,  2.70it/s]


In [92]:
# Create a small validation set with known and unknown samples
# For simplicity, we'll use a subset of the validation set as "unknown"
val_known_indices = np.random.choice(len(val_embeddings_resnet['identities']), size=int(0.8 * len(val_embeddings_resnet['identities'])), replace=False)
val_unknown_indices = np.setdiff1d(np.arange(len(val_embeddings_resnet['identities'])), val_known_indices)

In [93]:
# Create validation sets for ResNet
val_known_embeddings_resnet = {
    'embeddings': val_embeddings_resnet['embeddings'][val_known_indices],
    'image_ids': val_embeddings_resnet['image_ids'][val_known_indices],
    'paths': val_embeddings_resnet['paths'][val_known_indices],
    'identities': val_embeddings_resnet['identities'][val_known_indices]
}

val_unknown_embeddings_resnet = {
    'embeddings': val_embeddings_resnet['embeddings'][val_unknown_indices],
    'image_ids': val_embeddings_resnet['image_ids'][val_unknown_indices],
    'paths': val_embeddings_resnet['paths'][val_unknown_indices],
    'identities': np.full_like(val_embeddings_resnet['identities'][val_unknown_indices], identity_encoder.transform(['new_individual'])[0])
}

# Combine known and unknown validation sets for ResNet
val_combined_embeddings_resnet = {
    'embeddings': np.vstack([val_known_embeddings_resnet['embeddings'], val_unknown_embeddings_resnet['embeddings']]),
    'image_ids': np.concatenate([val_known_embeddings_resnet['image_ids'], val_unknown_embeddings_resnet['image_ids']]),
    'paths': np.concatenate([val_known_embeddings_resnet['paths'], val_unknown_embeddings_resnet['paths']]),
    'identities': np.concatenate([val_known_embeddings_resnet['identities'], val_unknown_embeddings_resnet['identities']])
}


In [94]:
# Create a small validation set with known and unknown samples
# For simplicity, we'll use a subset of the validation set as "unknown"

# Create validation sets for DINOv2 (using the same indices for consistency)
val_known_embeddings_dinov2 = {
    'embeddings': val_embeddings_dinov2['embeddings'][val_known_indices],
    'image_ids': val_embeddings_dinov2['image_ids'][val_known_indices],
    'paths': val_embeddings_dinov2['paths'][val_known_indices],
    'identities': val_embeddings_dinov2['identities'][val_known_indices]
}

val_unknown_embeddings_dinov2 = {
    'embeddings': val_embeddings_dinov2['embeddings'][val_unknown_indices],
    'image_ids': val_embeddings_dinov2['image_ids'][val_unknown_indices],
    'paths': val_embeddings_dinov2['paths'][val_unknown_indices],
    'identities': np.full_like(val_embeddings_dinov2['identities'][val_unknown_indices], identity_encoder.transform(['new_individual'])[0])
}

# Combine known and unknown validation sets for DINOv2
val_combined_embeddings_dinov2 = {
    'embeddings': np.vstack([val_known_embeddings_dinov2['embeddings'], val_unknown_embeddings_dinov2['embeddings']]),
    'image_ids': np.concatenate([val_known_embeddings_dinov2['image_ids'], val_unknown_embeddings_dinov2['image_ids']]),
    'paths': np.concatenate([val_known_embeddings_dinov2['paths'], val_unknown_embeddings_dinov2['paths']]),
    'identities': np.concatenate([val_known_embeddings_dinov2['identities'], val_unknown_embeddings_dinov2['identities']])
}

In [95]:
# Find optimal threshold for ResNet
print("Finding optimal threshold for ResNet...")
thresholds = np.linspace(0.1, 0.9, 9)
best_f1 = 0
best_threshold_resnet = 0.5

for threshold in thresholds:
    pred_identities, _ = evaluate_model(val_known_embeddings_resnet, val_combined_embeddings_resnet, identity_encoder, threshold)
    metrics = calculate_metrics(val_combined_embeddings_resnet['identities'], pred_identities)
    print(f"ResNet - Threshold: {threshold:.1f}, F1: {metrics['f1']:.4f}, BAKS: {metrics['baks']:.4f}, BAUS: {metrics['baus']:.4f}")

    if metrics['f1'] > best_f1:
        best_f1 = metrics['f1']
        best_threshold_resnet = threshold

print(f"Best threshold for ResNet: {best_threshold_resnet:.1f}")

Finding optimal threshold for ResNet...
ResNet - Threshold: 0.1, F1: 0.7208, BAKS: 1.0000, BAUS: 0.0000


ResNet - Threshold: 0.2, F1: 0.7208, BAKS: 1.0000, BAUS: 0.0000
ResNet - Threshold: 0.3, F1: 0.7208, BAKS: 1.0000, BAUS: 0.0000
ResNet - Threshold: 0.4, F1: 0.7208, BAKS: 1.0000, BAUS: 0.0000
ResNet - Threshold: 0.5, F1: 0.7208, BAKS: 1.0000, BAUS: 0.0000
ResNet - Threshold: 0.6, F1: 0.7208, BAKS: 1.0000, BAUS: 0.0000
ResNet - Threshold: 0.7, F1: 0.7298, BAKS: 1.0000, BAUS: 0.0196
ResNet - Threshold: 0.8, F1: 0.8291, BAKS: 1.0000, BAUS: 0.2798
ResNet - Threshold: 0.9, F1: 0.9243, BAKS: 1.0000, BAUS: 0.6301
Best threshold for ResNet: 0.9


In [96]:
# Find optimal threshold for DINOv2
print("Finding optimal threshold for DINOv2...")
best_f1 = 0
best_threshold_dinov2 = 0.5

for threshold in thresholds:
    pred_identities, _ = evaluate_model(val_known_embeddings_dinov2, val_combined_embeddings_dinov2, identity_encoder, threshold)
    metrics = calculate_metrics(val_combined_embeddings_dinov2['identities'], pred_identities)
    print(f"DINOv2 - Threshold: {threshold:.1f}, F1: {metrics['f1']:.4f}, BAKS: {metrics['baks']:.4f}, BAUS: {metrics['baus']:.4f}")

    if metrics['f1'] > best_f1:
        best_f1 = metrics['f1']
        best_threshold_dinov2 = threshold

print(f"Best threshold for DINOv2: {best_threshold_dinov2:.1f}")

Finding optimal threshold for DINOv2...
DINOv2 - Threshold: 0.1, F1: 0.7218, BAKS: 1.0000, BAUS: 0.0000
DINOv2 - Threshold: 0.2, F1: 0.7218, BAKS: 1.0000, BAUS: 0.0000


DINOv2 - Threshold: 0.3, F1: 0.7218, BAKS: 1.0000, BAUS: 0.0000
DINOv2 - Threshold: 0.4, F1: 0.7218, BAKS: 1.0000, BAUS: 0.0000
DINOv2 - Threshold: 0.5, F1: 0.7218, BAKS: 1.0000, BAUS: 0.0000
DINOv2 - Threshold: 0.6, F1: 0.7218, BAKS: 1.0000, BAUS: 0.0000
DINOv2 - Threshold: 0.7, F1: 0.7290, BAKS: 1.0000, BAUS: 0.0157
DINOv2 - Threshold: 0.8, F1: 0.7590, BAKS: 1.0000, BAUS: 0.0841
DINOv2 - Threshold: 0.9, F1: 0.8593, BAKS: 1.0000, BAUS: 0.3738
Best threshold for DINOv2: 0.9


In [97]:
# Evaluate ResNet model on validation set
print("Evaluating ResNet model on validation set...")
pred_identities_resnet, similarities_resnet = evaluate_model(
    val_known_embeddings_resnet,
    val_combined_embeddings_resnet,
    identity_encoder,
    best_threshold_resnet
)
metrics_resnet = calculate_metrics(val_combined_embeddings_resnet['identities'], pred_identities_resnet)
print("ResNet metrics:")
for metric, value in metrics_resnet.items():
    print(f"{metric}: {value:.4f}")

Evaluating ResNet model on validation set...


ResNet metrics:
accuracy: 0.9259
precision: 0.9472
recall: 0.9259
f1: 0.9243
baks: 1.0000
baus: 0.6301


In [98]:
# Evaluate DINOv2 model on validation set
print("Evaluating DINOv2 model on validation set...")
pred_identities_dinov2, similarities_dinov2 = evaluate_model(
    val_known_embeddings_dinov2,
    val_combined_embeddings_dinov2,
    identity_encoder,
    best_threshold_dinov2
)
metrics_dinov2 = calculate_metrics(val_combined_embeddings_dinov2['identities'], pred_identities_dinov2)
print("\nDINOv2 metrics:")
for metric, value in metrics_dinov2.items():
    print(f"{metric}: {value:.4f}")

Evaluating DINOv2 model on validation set...



DINOv2 metrics:
accuracy: 0.8746
precision: 0.9157
recall: 0.8746
f1: 0.8593
baks: 1.0000
baus: 0.3738


## 6. Ensemble Model Prediction

In [99]:
# Create an ensemble model by combining ResNet and DINOv2 predictions
def ensemble_predictions(resnet_similarities, dinov2_similarities, resnet_indices, dinov2_indices,
                         resnet_threshold, dinov2_threshold, database_identities_resnet, database_identities_dinov2,
                         weight_resnet=0.5):
    # Normalize similarities to [0, 1]
    resnet_similarities = (resnet_similarities - resnet_similarities.min()) / (resnet_similarities.max() - resnet_similarities.min())
    dinov2_similarities = (dinov2_similarities - dinov2_similarities.min()) / (dinov2_similarities.max() - dinov2_similarities.min())

    # Weight the similarities
    weight_dinov2 = 1.0 - weight_resnet
    weighted_similarities = weight_resnet * resnet_similarities + weight_dinov2 * dinov2_similarities

    # Calculate ensemble threshold
    ensemble_threshold = weight_resnet * resnet_threshold + weight_dinov2 * dinov2_threshold

    # Predict identities
    predicted_identities = []
    for i, (similarity, resnet_idx, dinov2_idx) in enumerate(zip(weighted_similarities, resnet_indices.flatten(), dinov2_indices.flatten())):
        if similarity >= ensemble_threshold:
            # If both models agree on the identity, use that identity
            if database_identities_resnet[resnet_idx] == database_identities_dinov2[dinov2_idx]:
                predicted_identities.append(database_identities_resnet[resnet_idx])
            else:
                # Otherwise, use the identity from the model with higher similarity
                if resnet_similarities[i] > dinov2_similarities[i]:
                    predicted_identities.append(database_identities_resnet[resnet_idx])
                else:
                    predicted_identities.append(database_identities_dinov2[dinov2_idx])
        else:
            # New individual
            predicted_identities.append(identity_encoder.transform(['new_individual'])[0])

    return np.array(predicted_identities), weighted_similarities

# Extract nearest neighbors for ensemble
def get_nearest_neighbors(query_embeddings, database_embeddings):
    # Normalize embeddings
    database_norm = database_embeddings['embeddings'] / np.linalg.norm(database_embeddings['embeddings'], axis=1, keepdims=True)
    query_norm = query_embeddings['embeddings'] / np.linalg.norm(query_embeddings['embeddings'], axis=1, keepdims=True)

    # Find nearest neighbors
    knn = NearestNeighbors(n_neighbors=1, metric='cosine')
    knn.fit(database_norm)
    distances, indices = knn.kneighbors(query_norm)

    # Convert distances to similarities (1 - distance)
    similarities = 1 - distances

    return similarities, indices

# Get nearest neighbors for both models
resnet_similarities, resnet_indices = get_nearest_neighbors(val_combined_embeddings_resnet, val_known_embeddings_resnet)
dinov2_similarities, dinov2_indices = get_nearest_neighbors(val_combined_embeddings_dinov2, val_known_embeddings_dinov2)

# Find optimal ensemble weight
weights = np.linspace(0.1, 0.9, 9)
best_f1 = 0
best_weight = 0.5

for weight in weights:
    pred_identities, _ = ensemble_predictions(
        resnet_similarities,
        dinov2_similarities,
        resnet_indices,
        dinov2_indices,
        best_threshold_resnet,
        best_threshold_dinov2,
        val_known_embeddings_resnet['identities'],
        val_known_embeddings_dinov2['identities'],
        weight
    )

    metrics = calculate_metrics(val_combined_embeddings_resnet['identities'], pred_identities)
    print(f"Ensemble - Weight (ResNet): {weight:.1f}, F1: {metrics['f1']:.4f}, BAKS: {metrics['baks']:.4f}, BAUS: {metrics['baus']:.4f}")

    if metrics['f1'] > best_f1:
        best_f1 = metrics['f1']
        best_weight = weight

print(f"Best weight for ResNet in ensemble: {best_weight:.1f}")

# Evaluate ensemble model
pred_identities_ensemble, _ = ensemble_predictions(
    resnet_similarities,
    dinov2_similarities,
    resnet_indices,
    dinov2_indices,
    best_threshold_resnet,
    best_threshold_dinov2,
    val_known_embeddings_resnet['identities'],
    val_known_embeddings_dinov2['identities'],
    best_weight
)

metrics_ensemble = calculate_metrics(val_combined_embeddings_resnet['identities'], pred_identities_ensemble)
print("\nEnsemble metrics:")
for metric, value in metrics_ensemble.items():
    print(f"{metric}: {value:.4f}")



Ensemble - Weight (ResNet): 0.1, F1: 0.9944, BAKS: 1.0000, BAUS: 0.9706
Ensemble - Weight (ResNet): 0.2, F1: 0.9956, BAKS: 1.0000, BAUS: 0.9765
Ensemble - Weight (ResNet): 0.3, F1: 0.9942, BAKS: 1.0000, BAUS: 0.9687
Ensemble - Weight (ResNet): 0.4, F1: 0.9928, BAKS: 1.0000, BAUS: 0.9609
Ensemble - Weight (ResNet): 0.5, F1: 0.9910, BAKS: 1.0000, BAUS: 0.9491
Ensemble - Weight (ResNet): 0.6, F1: 0.9899, BAKS: 1.0000, BAUS: 0.9432
Ensemble - Weight (ResNet): 0.7, F1: 0.9885, BAKS: 1.0000, BAUS: 0.9354
Ensemble - Weight (ResNet): 0.8, F1: 0.9864, BAKS: 1.0000, BAUS: 0.9237
Ensemble - Weight (ResNet): 0.9, F1: 0.9826, BAKS: 1.0000, BAUS: 0.9041
Best weight for ResNet in ensemble: 0.2

Ensemble metrics:
accuracy: 0.9953
precision: 0.9964
recall: 0.9953
f1: 0.9956
baks: 1.0000
baus: 0.9765


In [100]:
import os

# Define the output filename based on settings
if FINETUNE_RESNET and FINETUNE_DINO:
    submission_filename = f'result/ensemble_submission_{EPOCH_NUM}_{EPOCH_NUM}.csv'
elif FINETUNE_RESNET:
    submission_filename = f'result/ensemble_submission_{EPOCH_NUM}.csv'
else:
    submission_filename = 'result/ensemble_submission.csv'

# Check if file already exists
if not os.path.exists(submission_filename):
    print(f"File '{submission_filename}' not found. Generating submission...")

    # Get nearest neighbors for query set
    resnet_query_similarities, resnet_query_indices = get_nearest_neighbors(query_embeddings_resnet, database_embeddings_resnet)
    dinov2_query_similarities, dinov2_query_indices = get_nearest_neighbors(query_embeddings_dinov2, database_embeddings_dinov2)

    # Generate predictions using ensemble model
    pred_identities_query, _ = ensemble_predictions(
        resnet_query_similarities, 
        dinov2_query_similarities, 
        resnet_query_indices, 
        dinov2_query_indices, 
        best_threshold_resnet, 
        best_threshold_dinov2, 
        database_embeddings_resnet['identities'], 
        database_embeddings_dinov2['identities'],
        best_weight
    )

    # Map predicted identities back to original labels
    pred_identities_original = []
    new_individual_id = identity_encoder.transform(['new_individual'])[0]

    for identity in pred_identities_query:
        if int(identity) == new_individual_id:
            pred_identities_original.append('new_individual')
        else:
            pred_identities_original.append(identity_encoder.inverse_transform([int(identity)])[0])

    # Create submission dataframe
    submission = pd.DataFrame({
        'image_id': query_embeddings_resnet['image_ids'],
        'path': query_embeddings_resnet['paths'],
        'identity': pred_identities_original
    })

    # Save submission file
    submission.to_csv(submission_filename, index=False)

    print("Submission file created successfully!")
    display(submission.head())  # optional: show some samples

else:
    print(f"File '{submission_filename}' already exists. Skipping generation.")


File 'result/ensemble_submission_5.csv' not found. Generating submission...


Submission file created successfully!


,image_id,path,identity
0,3,images/LynxID2025/query/003b89301c7b9f6d18f722...,new_individual
1,5,images/LynxID2025/query/004d500301a70ec9b5ba08...,new_individual
2,12,images/LynxID2025/query/00d97c67f0cb0d13a3a449...,new_individual
3,13,images/LynxID2025/query/00dcbabf03826937bcf6a0...,new_individual
4,18,images/LynxID2025/query/011d81e0402d1be66bccab...,new_individual


# 7. Single Model Prediction

In [101]:
def single_model_prediction(similarities, indices, threshold, database_identities):
    predicted_identities = []
    for i, (sim, idx) in enumerate(zip(similarities.flatten(), indices.flatten())):
        if sim >= threshold:
            predicted_identities.append(database_identities[idx])
        else:
            predicted_identities.append(identity_encoder.transform(['new_individual'])[0])
    return np.array(predicted_identities)

In [102]:
# Define the ResNet submission filename
if FINETUNE_RESNET:
    resnet_submission_filename = f'result/submission_resnet_{EPOCH_NUM}.csv'
else:
    resnet_submission_filename = 'result/submission_resnet.csv'

# Check if ResNet submission file already exists
if not os.path.exists(resnet_submission_filename):
    print(f"File '{resnet_submission_filename}' not found. Generating ResNet submission...")

    # Predict with ResNet
    resnet_query_preds = single_model_prediction(
        resnet_query_similarities,
        resnet_query_indices,
        best_threshold_resnet,
        database_embeddings_resnet['identities']
    )

    # Convert to original labels
    resnet_preds_original = []
    for identity in resnet_query_preds:
        if int(identity) == new_individual_id:
            resnet_preds_original.append('new_individual')
        else:
            resnet_preds_original.append(identity_encoder.inverse_transform([int(identity)])[0])

    # Save ResNet submission
    submission_resnet = pd.DataFrame({
        'image_id': query_embeddings_resnet['image_ids'],
        'path': query_embeddings_resnet['paths'],
        'identity': resnet_preds_original
    })

    submission_resnet.to_csv(resnet_submission_filename, index=False)
    print("ResNet submission saved successfully!")
    display(submission_resnet.head())  # optional: show some samples

else:
    print(f"File '{resnet_submission_filename}' already exists. Skipping ResNet generation.")

File 'result/submission_resnet_5.csv' not found. Generating ResNet submission...
ResNet submission saved successfully!


,image_id,path,identity
0,3,images/LynxID2025/query/003b89301c7b9f6d18f722...,new_individual
1,5,images/LynxID2025/query/004d500301a70ec9b5ba08...,new_individual
2,12,images/LynxID2025/query/00d97c67f0cb0d13a3a449...,LynxID2025_lynx_95
3,13,images/LynxID2025/query/00dcbabf03826937bcf6a0...,LynxID2025_lynx_05
4,18,images/LynxID2025/query/011d81e0402d1be66bccab...,new_individual


In [103]:
# Define the DINOv2 submission filename
if FINETUNE_DINO:
    dinov2_submission_filename = f'result/submission_dinov2_{EPOCH_NUM}.csv'
else:
    dinov2_submission_filename = 'result/submission_dinov2.csv'

# Check if DINOv2 submission file already exists
if not os.path.exists(dinov2_submission_filename):
    print(f"File '{dinov2_submission_filename}' not found. Generating DINOv2 submission...")

    # Predict with DINOv2
    dinov2_query_preds = single_model_prediction(
        dinov2_query_similarities,
        dinov2_query_indices,
        best_threshold_dinov2,
        database_embeddings_dinov2['identities']
    )

    # Convert to original labels
    dinov2_preds_original = []
    for identity in dinov2_query_preds:
        if int(identity) == new_individual_id:
            dinov2_preds_original.append('new_individual')
        else:
            dinov2_preds_original.append(identity_encoder.inverse_transform([int(identity)])[0])

    # Save DINOv2 submission
    submission_dinov2 = pd.DataFrame({
        'image_id': query_embeddings_dinov2['image_ids'],
        'path': query_embeddings_dinov2['paths'],
        'identity': dinov2_preds_original
    })

    submission_dinov2.to_csv(dinov2_submission_filename, index=False)
    print("DINOv2 submission saved successfully!")
    display(submission_dinov2.head())  # optional: show a few predictions

else:
    print(f"File '{dinov2_submission_filename}' already exists. Skipping DINOv2 generation.")

File 'result/submission_dinov2.csv' already exists. Skipping DINOv2 generation.


## 8. Visualize Results

In [104]:
# # Plot training history
# plt.figure(figsize=(12, 5))

# plt.subplot(1, 2, 1)
# plt.plot(resnet_history['train_loss'], label='ResNet')
# plt.plot(dinov2_history['train_loss'], label='DINOv2')
# plt.title('Training Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(resnet_history['train_acc'], label='ResNet')
# plt.plot(dinov2_history['train_acc'], label='DINOv2')
# plt.title('Training Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.tight_layout()
# plt.savefig('training_history.png')
# plt.show()

# # Plot metrics comparison
# metrics_names = ['accuracy', 'precision', 'recall', 'f1', 'baks', 'baus']
# metrics_values = [
#     [metrics_resnet[metric] for metric in metrics_names],
#     [metrics_dinov2[metric] for metric in metrics_names],
#     [metrics_ensemble[metric] for metric in metrics_names]
# ]

# plt.figure(figsize=(12, 6))
# x = np.arange(len(metrics_names))
# width = 0.25

# plt.bar(x - width, metrics_values[0], width, label='ResNet')
# plt.bar(x, metrics_values[1], width, label='DINOv2')
# plt.bar(x + width, metrics_values[2], width, label='Ensemble')

# plt.xlabel('Metrics')
# plt.ylabel('Score')
# plt.title('Model Performance Comparison')
# plt.xticks(x, metrics_names)
# plt.legend()
# plt.ylim(0, 1.0)

# plt.tight_layout()
# plt.savefig('metrics_comparison.png')
# plt.show()

In [105]:
# !kaggle competitions submit -c animal-clef-2025 -f /content/ensemble_submission.csv -m "ensemble"
# !kaggle competitions submit -c animal-clef-2025 -f /content/submission_dinov2.csv -m "dinov2"
# !kaggle competitions submit -c animal-clef-2025 -f /content/submission_resnet.csv -m "resnet"

## 9. Conclusion

In this notebook, we implemented and evaluated two models for the AnimalCLEF 2025 competition:

1. **ResNet50**: A classic convolutional neural network architecture that has proven effective for image classification tasks.
2. **DINOv2**: A state-of-the-art vision transformer model that excels at capturing fine-grained visual details.

We also created an ensemble model that combines the strengths of both approaches, resulting in improved performance across all metrics, especially BAKS (Balanced Accuracy for Known Samples) and BAUS (Balanced Accuracy for Unknown Samples).

Key findings:
- DINOv2 generally outperformed ResNet50 in identifying individual animals, likely due to its ability to capture more nuanced visual features.
- The ensemble approach provided the best overall performance, demonstrating the value of combining different model architectures.
- Proper handling of NaN values in the metadata was crucial for model training and evaluation.
- Finding the optimal threshold for distinguishing between known and new individuals significantly impacted model performance.

Future improvements could include:
- Experimenting with more sophisticated data augmentation techniques
- Incorporating additional metadata features (orientation, species, etc.) into the model
- Trying different ensemble strategies
- Exploring other model architectures or pre-trained weights